In [ ]:
# set random seed
import random
random.seed(42)
import numpy as np
np.random.seed(42)
import torch
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Generate Data

In [ ]:
import pyepo
# generate data
grid = (5,5) # grid size
num_data = 100 # number of training data
num_test = 1000
num_feat = 5 # size of feature
deg = 4 # polynomial degree
e = 0.5 # noise width
feats, costs = pyepo.data.shortestpath.genData(num_data+num_test, num_feat, grid, deg, e, seed=42)

# Build Model

In [ ]:
# build optModel
from pyepo.model.grb import optGrbModel

class shortestPathModel(optGrbModel):

    def __init__(self):
        self.grid = (5,5)
        self.arcs = self._getArcs()
        super().__init__()

    def _getArcs(self):
        """
        A helper method to get list of arcs for grid network

        Returns:
            list: arcs
        """
        arcs = []
        for i in range(self.grid[0]):
            # edges on rows
            for j in range(self.grid[1] - 1):
                v = i * self.grid[1] + j
                arcs.append((v, v + 1))
            # edges in columns
            if i == self.grid[0] - 1:
                continue
            for j in range(self.grid[1]):
                v = i * self.grid[1] + j
                arcs.append((v, v + self.grid[1]))
        return arcs

    def _getModel(self):
        """
        A method to build Gurobi model

        Returns:
            tuple: optimization model and variables
        """
        import gurobipy as gp
        from gurobipy import GRB
        # ceate a model
        m = gp.Model("shortest path")
        # varibles
        x = m.addVars(self.arcs, name="x")
        # sense
        m.modelSense = GRB.MINIMIZE
        # flow conservation constraints
        for i in range(self.grid[0]):
            for j in range(self.grid[1]):
                v = i * self.grid[1] + j
                expr = 0
                for e in self.arcs:
                    # flow in
                    if v == e[1]:
                        expr += x[e]
                    # flow out
                    elif v == e[0]:
                        expr -= x[e]
                # source
                if i == 0 and j == 0:
                    m.addConstr(expr == -1)
                # sink
                elif i == self.grid[0] - 1 and j == self.grid[0] - 1:
                    m.addConstr(expr == 1)
                # transition
                else:
                    m.addConstr(expr == 0)
        return m, x

# Prepara training and test data

In [ ]:
# split train test data
from sklearn.model_selection import train_test_split
x_train, x_test, c_train, c_test = train_test_split(feats, costs, test_size=num_test, random_state=42)

In [ ]:
optmodel = shortestPathModel()

In [ ]:
# get optDataset
dataset_train = pyepo.data.dataset.optDataset(optmodel, x_train, c_train)
dataset_test = pyepo.data.dataset.optDataset(optmodel, x_test, c_test)

In [ ]:
# set data loader
from torch.utils.data import DataLoader
batch_size = 20
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=False)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

# Linear regression

In [ ]:
from torch import nn

# build linear model
class LinearRegression(nn.Module):

    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(num_feat, (grid[0]-1)*grid[1]+(grid[1]-1)*grid[0])

    def forward(self, x):
        out = self.linear(x)
        return out

## Initial the predictor

In [ ]:
import torch
# init model
reg = LinearRegression()

# Evaluation

In [ ]:
import pyepo
regret = pyepo.metric.regret(reg, optmodel, loader_test)

# Training different models

In [ ]:
import time

# train model
def trainModel(reg, loss_func, method_name, num_epochs=20, lr=1e-2):
    # set adam optimizer
    optimizer = torch.optim.Adam(reg.parameters(), lr=lr)
    # train mode
    reg.train()
    # init log
    loss_log = []
    loss_log_regret = [pyepo.metric.regret(reg, optmodel, loader_test)]
    # print("epoch = ",0,", regret = ",loss_log_regret[0])
    # init elpased time
    elapsed = 0
    for epoch in range(num_epochs):
        # start timing
        tick = time.time()
        # load data
        for i, data in enumerate(loader_train):
            x, c, w, z = data
            # cuda
            if torch.cuda.is_available():
                x, c, w, z = x.cuda(), c.cuda(), w.cuda(), z.cuda()
            # forward pass
            cp = reg(x)
            if method_name == "spo+":
                loss = loss_func(cp, c, w, z)
            if method_name in ["ptb", "pfy", "imle", "aimle", "nce", "cmap"]:
                loss = loss_func(cp, w)
            if method_name in ["dbb", "nid"]:
                loss = loss_func(cp, c, z)
            if method_name in ["pg", "ltr"]:
                loss = loss_func(cp, c)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # record time
            tock = time.time()
            elapsed += tock - tick
            # log
            loss_log.append(loss.item())
        regret = pyepo.metric.regret(reg, optmodel, loader_test)
        print("epoch = ",epoch,", regret = ",loss_log_regret[0])
        loss_log_regret.append(regret)
        print("Epoch {:2},  Loss: {:9.4f},  Regret: {:7.4f}%".format(epoch+1, loss.item(), regret*100))
    print("Total Elapsed Time: {:.2f} Sec.".format(elapsed))
    return loss_log, loss_log_regret

# SPO+

In [ ]:
import torch
# init model
reg = LinearRegression()
# cuda
if torch.cuda.is_available():
    reg = reg.cuda()

## Initialize

In [ ]:
# init SPO+ loss
spop = pyepo.func.SPOPlus(optmodel, processes=2)

# Training

In [ ]:
loss_log, loss_log_regret = trainModel(reg, loss_func=spop, method_name="spo+")

# Check correctness

In [ ]:
arcs_arr = optmodel.arcs
def obtain_path(arcs_arr,sol):
    path_arr = []
    for arc_index in range(len(arcs_arr)):
        if sol[arc_index] > 0:
            path_arr.append(arcs_arr[arc_index])
    return path_arr

In [ ]:
def getArcs(grid):
    arcs = []
    for i in range(grid[0]):
        # edges on rows
        for j in range(grid[1] - 1):
            v = i * grid[1] + j
            arcs.append((v, v + 1))
        # edges in columns
        if i == grid[0] - 1:
            continue
        for j in range(grid[1]):
            v = i * grid[1] + j
            arcs.append((v, v + grid[1]))
    return arcs

def solve_Shortest_Path(arcs,cost):
    """
    A method to build Gurobi model

    Returns:
        tuple: optimization model and variables
    """
    import gurobipy as gp
    from gurobipy import GRB
    # ceate a model
    m = gp.Model("shortest path")
    m.setParam('OutputFlag', 0)
    # varibles
    x = m.addVars(arcs, name="x")
    # sense
    # m.modelSense = GRB.MINIMIZE
    # flow conservation constraints
    for i in range(grid[0]):
        for j in range(grid[1]):
            v = i * grid[1] + j
            expr = 0
            for e in arcs:
                # flow in
                if v == e[1]:
                    expr += x[e]
                # flow out
                elif v == e[0]:
                    expr -= x[e]
            # source
            if i == 0 and j == 0:
                m.addConstr(expr == -1)
            # sink
            elif i == grid[0] - 1 and j == grid[0] - 1:
                m.addConstr(expr == 1)
            # transition
            else:
                m.addConstr(expr == 0)
    m.setObjective( sum([cost[ind] * x[arcs_arr[ind]] for ind in range(len(arcs_arr))]) , GRB.MINIMIZE)
    m.optimize()
    sol = m.getAttr('x')
    # print("sol = ",sol)
    shortest_path = obtain_path(arcs_arr,sol)
    
    obj = m.getObjective().getValue()
    # print("obj = ",obj,"shortest_path = ",shortest_path)
    return obj,sol

In [ ]:
from pyepo import EPO
def regret(predmodel, optmodel, dataloader):
    """
    A function to evaluate model performance with normalized true regret

    Args:
        predmodel (nn): a regression neural network for cost prediction
        optmodel (optModel): an PyEPO optimization model
        dataloader (DataLoader): Torch dataloader from optDataSet

    Returns:
        float: true regret loss
    """
    # evaluate
    predmodel.eval()
    loss = 0
    optsum = 0
    # load data
    iterations = 1
    for data in dataloader:
        x, c, w, z = data
        # cuda
        if next(predmodel.parameters()).is_cuda:
            x, c, w, z = x.cuda(), c.cuda(), w.cuda(), z.cuda()
            print("CUDA")
        # predict
        with torch.no_grad(): # no grad
            cp = predmodel(x).to("cpu").detach().numpy()
            # print("itereations = ",iterations,",Hello World")
        # solve
        for j in range(cp.shape[0]):
            # accumulate loss
            loss_tem,sol = calRegret(optmodel, cp[j], c[j].to("cpu").detach().numpy(),
                              z[j].item())
            path_dict = obtain_path(arcs_arr,sol)
            # print("j = ",j)
            # print("x = ",x)
            # print("predict cost = ",cp[j])
            # print("sol = ",sol)
            # print("ShortestPath = ",path_dict)
            # print("loss = ",loss_tem)
            # print()
            loss = loss + loss_tem
        optsum += abs(z).sum().item()
        iterations = iterations + 1
    # turn back train mode
    predmodel.train()
    # normalized
    return loss / (optsum + 1e-7)


def calRegret(optmodel, pred_cost, true_cost, true_obj):
    """
    A function to calculate normalized true regret for a batch

    Args:
        optmodel (optModel): optimization model
        pred_cost (torch.tensor): predicted costs
        true_cost (torch.tensor): true costs
        true_obj (torch.tensor): true optimal objective values

    Returns:predmodel
        float: true regret losses
    """
    # opt sol for pred cost
    optmodel.setObj(pred_cost)
    sol, _ = optmodel.solve()
    # obj with true cost
    obj = np.dot(sol, true_cost)
    # loss
    if optmodel.modelSense == EPO.MINIMIZE:
        loss = obj - true_obj
    if optmodel.modelSense == EPO.MAXIMIZE:
        loss = true_obj - obj
    return loss,sol


In [ ]:
# set adam optimizer
optimizer = torch.optim.Adam(reg.parameters(), lr=1e-2)
# train mode
reg.train()
# init log
loss_log = []
regret(reg, optmodel, loader_test)

In [ ]:
# set adam optimizer
optimizer = torch.optim.Adam(reg.parameters(), lr=1e-2)
# train mode
reg.train()
# init log
loss_log = []
loss_log_regret = [pyepo.metric.regret(reg, optmodel, loader_test)]

In [ ]:
regret(reg, optmodel, loader_test)

In [ ]:
cost_oracle = []
cost_pred_prev = []
for i, data in enumerate(loader_test):
    x, c, w, z = data
    cost_predict = reg(x)
    
    sample_length = np.shape(c)[0]
    # print("sample_length = ",sample_length)
    w_all = w.numpy()
    cost_true = c.numpy()
    x_all = x.numpy()
    print("z = ",z.numpy()[:,0])
    cost_true_arr = np.zeros(sample_length)
    for sample_index in range(sample_length):
        # print("======= sample index = ",sample_index,"=============")
        # print("w Shortest_Path = ",obtain_path(arcs_arr,w_all[sample_index]))
        obj_true,sol_true = solve_Shortest_Path(arcs_arr,cost_true[sample_index])
        cost_true_arr[sample_index] = obj_true
    print("obj_true = ",cost_true_arr)
        # obj_pred,sol_pred = solve_Shortest_Path(arcs_arr,cost_predict[sample_index])
        # cost_oracle.append(obj_true)
        # cost_pred_prev.append(obj_pred)

In [ ]:
np.mean(cost_pred_prev)

In [ ]:
import time
# init elpased time
elapsed = 0
for epoch in range(2):
    # start timing
    tick = time.time()
    # load data
    for i, data in enumerate(loader_train):
        x, c, w, z = data
        # sample_length = np.shape(c)[0]
        # w_all = w.numpy()
        # cost_all = c.numpy()
        # for sample_index in range(1):
        #     cost = cost_all[sample_index]
        #     print("======= sample index = ",sample_index,"=============")
        #     print("w Shortest_Path = ",obtain_path(arcs_arr,w_all[sample_index]))
        #     solve_Shortest_Path(arcs_arr,cost)
            
        # cuda
        if torch.cuda.is_available():
            x, c, w, z = x.cuda(), c.cuda(), w.cuda(), z.cuda()
        # print("x = ",x)
        # forward pass
        cp = reg(x)

        loss = spop(cp, c, w, z)

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # record time
        tock = time.time()
        elapsed += tock - tick
        # log
        loss_log.append(loss.item())
        regret(reg, optmodel, loader_test)
